In [ ]:
from dreye.core import Spectrum, MeasuredSpectraContainer, \
    MeasuredSpectrum, convert_measurement, CalibrationSpectrum, \
    AbstractSpectrum, Signal, Domain
from dreye.constants import UREG
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
np.random.seed(10)

In [ ]:
from dreye.core import create_gaussian_spectrum

In [ ]:
from dreye.core import create_measured_spectra, create_measured_spectrum

In [ ]:
from dreye.utilities import is_listlike

In [ ]:
wl = np.arange(300, 600)
s = norm.pdf(wl, 450, 10)
s /= np.sum(s)
noise = 0.01
s_noise = s + np.random.normal(0, noise, size=s.shape)

wl is a numpy array of the desired wavelengths
s is a gaussian distribution 
s_noise is a noisy version of the signal- if you are simulating measurements, you will simulate them with noise, so adding noise will basically simulate the imperfect measurements from the LEDs

In [ ]:
plt.plot(wl, s)

In [ ]:
plt.plot(wl, s)
plt.plot(wl, s_noise)

In [ ]:
gaussians = create_gaussian_spectrum(
    wl, 
    [340, 440, 540], 
    intensity=0.5, 
    background=np.ones(wl.shape), 
    add_background=True, 
    cdf=False, 
    filter=True,
)

Convenience function to create a bunch of gaussian spectra- could be for simulating LEDs, creating single wavelegnth stimuli, opsin sensitivities, etc- anything that needs a gaussian spectrum. 

MAKE A NEW NOTEBOOK JUST FOR THIS- centers = centers of dist, units- spectral photon flux and spectral irradiance ( can specify micro / nano, cdf-true means instead of plotting pdf it has cdf of those gaussians none= pdf, false plots 1-cdf, background = can pass numpy array of same length as wavelengths to add that background OR you can filter by that background instead of using them as pure spectral distributions, zero_cutoff= cuts off spectral distribution at 0 if there are negative values 

In [ ]:
create_gaussian_spectrum?

In [ ]:
plt.plot(
    gaussians.domain, gaussians, 
)

These gaussians are an instance of a class in the dreye package (here it is an arbitrary signals), here you see three signals, each are a low pass signal filtering out high frequencies. just to show how the convenience function works- doesnt have to be part of the tutorial itself. we can save the data and load it up in the tutorial as data- this is just dummy data 

In [ ]:
s2 = norm.pdf(wl[:, None], [300, 400, 500], 10)

In [ ]:
s2.shape

In [ ]:
np.arange(100).reshape(10, 10).mean(0)[None]

In [ ]:
cal = CalibrationSpectrum(
    np.ones(wl.shape), 
    wl, 
    area=1
)

Cal- you can load up a calibration spectrum as a dummy calibration like a flat spectral distribution- now you can create a measurement 

In [ ]:
float(asarray([1]))

In [ ]:
meas3 = create_measured_spectrum(
    np.arange(1, 10)[None, :] * norm.pdf(wl, 400, 10)[:, None], 
    np.arange(1, 10), 
    wl, 
    calibration=cal, 
    integration_time=1, 
)

This creates a measurement spectrum- allows you to hold one measurement across different voltages. you can pass this through the photoreceptor class to do fittings, etc. This is for testing - this notebook tells you how to fit 

In [ ]:
signal = AbstractSpectrum(
    s_noise, wl
)

In [ ]:
measurement2 = convert_measurement(
    AbstractSpectrum(
        s, wl
    ), cal, 1
).uE

for i in np.arange(2, 10):
    m = convert_measurement(
        AbstractSpectrum(
            i * s, 
            wl
        ),
        cal, 1
    ).irradiance
    
    measurement2 = measurement2.concat(
        m, 
    )

In [ ]:
plt.plot(measurement2.domain, measurement2)

In [ ]:
measurement = convert_measurement(
    signal, cal, 1
).uE

for i in np.arange(2, 10):
    m = convert_measurement(
        AbstractSpectrum(
            i * s + np.random.normal(0, noise, size=s.shape), 
            wl
        ),
        cal, 1
    ).irradiance
    
    measurement = measurement.concat(
        m, 
    )

In [ ]:
plt.plot(measurement.domain, measurement)

In [ ]:
measured_spectrum = MeasuredSpectrum(
    measurement,
    labels=Domain(np.arange(1, 10), units='volts'), 
    smoothing_window=25, 
)

In [ ]:
measured_spectrum2 = MeasuredSpectrum(
    measurement2,
    labels=Domain(np.arange(1, 10), units='volts'), 
    smoothing_window=25, 
)

In [ ]:
spm = measured_spectrum.to_measured_spectra(
    name='LED',
    zero_boundary=0, 
    zero_is_lower=True
)

print(spm.interpolator_kwargs)
print(spm)
print(spm.domain)

y_points = [0, 0.06, 0.1, 0.2]
x_points = spm.map(y_points)

In [ ]:
x_points

In [ ]:
spm.domain_bounds

In [ ]:
msp = measured_spectrum.smooth

In [ ]:
plt.plot(msp.domain, msp)


In [ ]:
spm2 = msp.to_measured_spectra(
    name='LED',
    zero_boundary=0, 
    zero_is_lower=True
)

x_points2 = spm2.map(y_points)

In [ ]:
spm2.zero_boundary

In [ ]:
spm3 = measured_spectrum2.to_measured_spectra(
    name='LED',
    zero_boundary=0, 
    zero_is_lower=True
)
x_points3 = spm3.map(y_points)

In [ ]:
plt.scatter(x_points, y_points)
plt.plot(spm.domain, spm)
plt.scatter(x_points2, y_points)
plt.plot(spm2.domain, spm2)
plt.scatter(x_points3, y_points)
plt.plot(spm3.domain, spm3)

In [ ]:
for idx, led in enumerate([300, 360, 400, 500]):
    
    s2 = norm.pdf(wl, led, 20)
    mess = convert_measurement(
        AbstractSpectrum(
            s2 + np.random.normal(0, noise, size=s.shape), 
            wl
        ), cal, 1
    ).uE

    for i in np.arange(2, 10):
        m = convert_measurement(
            AbstractSpectrum(
                i * s2 + np.random.normal(0, noise, size=s.shape), 
                wl
            ),
            cal, 1
        ).irradiance

        mess = mess.concat(
            m, 
        )

    ms = MeasuredSpectrum(
        mess,
        labels=Domain(np.arange(1, 10), units='volts'), 
        smoothing_window=25, 
    ).to_measured_spectra(
        name=f'LED{led}', 
        zero_boundary=0, 
        zero_is_lower=True
    )
        
    if idx == 0:
        mss = ms
    else:
        mss = mss.concat(ms)

In [ ]:
mss.shape

In [ ]:
plt.plot(mss.domain, mss)

In [ ]:
from dreye.core import LogPhotoreceptor, \
    RelativeOpsinSensitivity, Spectrum, Domain
from scipy.stats import norm

fitting arbitrary spectral distributions to PRs. Gonna have one notebook to run through dummy measurements (testAO) so you can test without hardware. and one would be fitting and how the fitting works 

Dummy AO measurement 
FITTING STARTS HERE ^
-there can be multiple notebooks for this 

In [ ]:
wl = np.arange(300, 600)
ops = norm.pdf(wl[:, None], np.array([340, 320, 433])[None, :], 40)

In [ ]:
opsin = RelativeOpsinSensitivity(
    ops, 
    domain=wl, 
)

In [ ]:
pr = LogPhotoreceptor(opsin)



In [ ]:
plt.plot(opsin.domain, opsin)

In [ ]:
ill = norm.pdf(wl, 400, 10) * 10000
bg = np.ones(wl.shape)
bg /= np.sum(bg) / 10000

In [ ]:
illuminant = Spectrum(
    ill, 
    domain=wl
).uE
background = Spectrum(
    bg, 
    domain=wl
).uE

In [ ]:
plt.plot(illuminant.domain, illuminant)
plt.plot(background.domain, background)

In [ ]:
background.integral, illuminant.integral

In [ ]:
from dreye.io import read_json
system_loaded = read_json('measurement_test.json')
mss = system_loaded.spms

In [ ]:
plt.plot(mss)

In [ ]:
targets, A = pr.get_qs(
    mss, 
    illuminant/20, 
    background=background, 
)

In [ ]:
bg_weights, bg_res, bg_new = mss.fit(
    background, return_fit=True, return_res=True
)

In [ ]:
bg_weights

In [ ]:
plt.plot(bg_new.domain, bg_new)
plt.plot(background.domain, background)

In [ ]:
print(A)

In [ ]:
plt.plot(illuminant.domain, illuminant/10 +  background)

In [ ]:
w, res = pr.fit(
    mss, 
    illuminant, 
    background=background, 
    return_res=True, 
    only_uniques=False
)

In [ ]:
pr.fit?

In [ ]:
res

In [ ]:
print(w)

In [ ]:
print(mss.map(w))

In [ ]:
w.shape

In [ ]:
mss.units

In [ ]:
print(w)

In [ ]:
mss.bounds

In [ ]:
map_values = np.random.random(
    (100, mss.shape[mss.other_axis])
) * mss.bounds[1][None, :]

In [ ]:
mapped = mss.map(map_values, method='isotonic')

In [ ]:
xmss = mss.map(w, top=10)
print(xmss)

In [ ]:
mss_smooth = mss.window_filter(
    3, 'savgol', polyorder=2
)
type(mss_smooth)

In [ ]:
xmss2 = mss_smooth.map(w)
print(xmss2)

In [ ]:
plt.plot(mss.domain, mss)

for ixms, iw in zip(np.squeeze(xmss), np.squeeze(w)):
    plt.scatter(
        ixms, iw, 
    )

In [ ]:
plt.plot(mss.domain, mss)

for ixms, iw in zip(mapped.T, map_values.T):
    plt.scatter(
        ixms, iw, 
    )

In [ ]:
mss.bounds

In [ ]:
mss.domain_bounds

In [ ]:
opsin2 = RelativeOpsinSensitivity(
    norm.pdf(wl[:,None], asarray([[330, 450, 560]]), 40),
    domain=wl, 
)

In [ ]:
pr = LogPhotoreceptor(opsin2)



In [ ]:
ill2 = Spectrum(
    norm.pdf(wl[:,None], asarray([[360, 480, 530, 320]]), 10), 
    domain=wl, 
    units='microspectralphotonflux'
)

In [ ]:
plt.plot(ill2)
#plt.plot(opsin2)

In [ ]:
plt.plot(opsin2)

In [ ]:
plt.plot(mss.normalized_spectrum)

In [ ]:
w, res = pr.fit(
    mss, 
    ill2/20 +  background, 
    background=background, 
    return_res=True, 
)

In [ ]:
print(w)
w.shape

In [ ]:
res[1].fun

In [ ]:
targets, A = pr.get_qs(
    mss, 
    ill2 +  background,  
    background=background, 
)

In [ ]:
targets

In [ ]:
print(A)

In [ ]:
xmss = mss.map(w)

In [ ]:
plt.plot(mss.domain, mss)

for ixms, iw in zip(np.squeeze(xmss.T), np.squeeze(w.T)):
    plt.scatter(
        ixms, iw, 
    )